In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sqlalchemy import create_engine
import csv
import pymongo
import json


In [2]:
# File to Load
file = "resources/flying_etiquette_survey.csv"

etiquette_df = pd.read_csv(file)
etiquette_df.head()

,RespondentID,How often do you travel by plane?,Do you ever recline your seat when you fly?,How tall are you?,Do you have any children under 18?,"In a row of three seats, who should get to use the two arm rests?","In a row of two seats, who should get to use the middle arm rest?",Who should have control over the window shade?,Is itrude to move to an unsold seat on a plane?,"Generally speaking, is it rude to say more than a few words tothe stranger sitting next to you on a plane?",...,Is itrude to wake a passenger up if you are trying to walk around?,"In general, is itrude to bring a baby on a plane?","In general, is it rude to knowingly bring unruly children on a plane?",Have you ever used personal electronics during take off or landing in violation of a flight attendant's direction?,Have you ever smoked a cigarette in an airplane bathroom when it was against the rules?,Gender,Age,Household Income,Education,Location (Census Region)
0,3436139758,Once a year or less,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3434278696,Once a year or less,About half the time,"6'3""",Yes,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",...,"No, not at all rude","No, not at all rude","No, not at all rude",No,No,Male,30-44,NaN,Graduate degree,Pacific
2,3434275578,Once a year or less,Usually,"5'8""",No,Whoever puts their arm on the arm rest first,The arm rests should be shared,The person in the window seat should have excl...,"No, not rude at all","No, not at all rude",...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",No,No,Male,30-44,"$100,000 - $149,999",Bachelor degree,Pacific
3,3434268208,Once a year or less,Always,"5'11""",No,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",No,No,Male,30-44,"$0 - $24,999",Bachelor degree,Pacific
4,3434250245,Once a month or less,About half the time,"5'7""",No,The person in the middle seat gets both arm rests,The person in aisle,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",Yes,No,Male,30-44,"$50,000 - $99,999",Bachelor degree,Pacific


In [3]:
etiquette_df.sort_values(by=["How often do you travel by plane?"])

,RespondentID,How often do you travel by plane?,Do you ever recline your seat when you fly?,How tall are you?,Do you have any children under 18?,"In a row of three seats, who should get to use the two arm rests?","In a row of two seats, who should get to use the middle arm rest?",Who should have control over the window shade?,Is itrude to move to an unsold seat on a plane?,"Generally speaking, is it rude to say more than a few words tothe stranger sitting next to you on a plane?",...,Is itrude to wake a passenger up if you are trying to walk around?,"In general, is itrude to bring a baby on a plane?","In general, is it rude to knowingly bring unruly children on a plane?",Have you ever used personal electronics during take off or landing in violation of a flight attendant's direction?,Have you ever smoked a cigarette in an airplane bathroom when it was against the rules?,Gender,Age,Household Income,Education,Location (Census Region)
202,3433240354,A few times per month,Never,"5'11""",Yes,The person in the middle seat gets both arm rests,The person by the window,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",...,"Yes, somewhat rude","No, not at all rude","Yes, somewhat rude",No,No,Male,45-60,"$100,000 - $149,999",Graduate degree,West South Central
492,3432712883,A few times per month,About half the time,"6'1""",No,The arm rests should be shared,The arm rests should be shared,The person in the window seat should have excl...,"No, not rude at all","No, not at all rude",...,"No, not at all rude","No, not at all rude","Yes, somewhat rude",Yes,No,Male,> 60,150000,Bachelor degree,Pacific
1020,3431744533,A few times per month,Once in a while,"5'4""",Yes,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",...,"Yes, somewhat rude","No, not at all rude","Yes, somewhat rude",No,No,Female,30-44,NaN,Graduate degree,Pacific
756,3432298537,A few times per month,Always,"5'6""",No,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","Yes, somewhat rude",...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",Yes,No,Male,45-60,"$100,000 - $149,999",Some college or Associate degree,South Atlantic
187,3433258577,A few times per month,Always,"5'8""",Yes,Whoever puts their arm on the arm rest first,Whoever puts their arm on the arm rest first,The person in the window seat should have excl...,"No, not rude at all","No, not at all rude",...,"Yes, somewhat rude","No, not at all rude","Yes, somewhat rude",No,No,Female,30-44,"$100,000 - $149,999",Bachelor degree,South Atlantic
161,3433279798,A few times per month,Once in a while,"6'4""",No,The person in the middle seat gets both arm rests,The person in aisle,The person in the window seat should have excl...,"No, not rude at all","No, not at all rude",...,"Yes, somewhat rude","No, not at all rude","No, not at all rude",No,No,Male,30-44,150000,Graduate degree,Pacific
162,3433278680,A few times per month,Usually,"5'4""",No,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","Yes, somewhat rude",...,"No, not at all rude","No, not at all rude","No, not at all rude",No,No,Female,> 60,"$100,000 - $149,999",Graduate degree,South Atlantic
630,3432499380,A few times per month,Never,"6'0""",No,Other (please specify),The arm rests should be shared,The person in the window seat should have excl...,"Yes, somewhat rude","No, not at all rude",...,"No, not at all rude","Yes, very rude","Yes, very rude",No,No,Male,45-60,"$100,000 - $149,999",Some college or Associate degree,South Atlantic
45,3433926438,A few times per month,About half the time,"6'2""",No,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",...,"No, not at all rude","No, not at all rud

In [4]:
etiquette_df[["How often do you travel by plane?", "Education"]].head()

,How often do you travel by plane?,Education
0,Once a year or less,NaN
1,Once a year or less,Graduate degree
2,Once a year or less,Bachelor degree
3,Once a year or less,Bachelor degree
4,Once a month or less,Bachelor degree


In [5]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [9]:
db = client.survey_db
collection = db.survey_metadata

In [12]:
etiquette_df.set_index('RespondentID')

,How often do you travel by plane?,Do you ever recline your seat when you fly?,How tall are you?,Do you have any children under 18?,"In a row of three seats, who should get to use the two arm rests?","In a row of two seats, who should get to use the middle arm rest?",Who should have control over the window shade?,Is itrude to move to an unsold seat on a plane?,"Generally speaking, is it rude to say more than a few words tothe stranger sitting next to you on a plane?","On a 6 hour flight from NYC to LA, how many times is it acceptable to get up if you're not in an aisle seat?",...,Is itrude to wake a passenger up if you are trying to walk around?,"In general, is itrude to bring a baby on a plane?","In general, is it rude to knowingly bring unruly children on a plane?",Have you ever used personal electronics during take off or landing in violation of a flight attendant's direction?,Have you ever smoked a cigarette in an airplane bathroom when it was against the rules?,Gender,Age,Household Income,Education,Location (Census Region)
RespondentID,,,,,,,,,,,,,,,,,,,,,
3436139758,Once a year or less,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3434278696,Once a year or less,About half the time,"6'3""",Yes,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",Twice,...,"No, not at all rude","No, not at all rude","No, not at all rude",No,No,Male,30-44,NaN,Graduate degree,Pacific
3434275578,Once a year or less,Usually,"5'8""",No,Whoever puts their arm on the arm rest first,The arm rests should be shared,The person in the window seat should have excl...,"No, not rude at all","No, not at all rude",Three times,...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",No,No,Male,30-44,"$100,000 - $149,999",Bachelor degree,Pacific
3434268208,Once a year or less,Always,"5'11""",No,The arm rests should be shared,The arm rests should be shared,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",Three times,...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",No,No,Male,30-44,"$0 - $24,999",Bachelor degree,Pacific
3434250245,Once a month or less,About half the time,"5'7""",No,The person in the middle seat gets both arm rests,The person in aisle,Everyone in the row should have some say,"No, not rude at all","No, not at all rude",Twice,...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",Yes,No,Male,30-44,"$50,000 - $99,999",Bachelor degree,Pacific
3434245875,Once a year or less,Usually,"5'9""",Yes,Whoever puts their arm on the arm rest first,Whoever puts their arm on the arm rest first,The person in the window seat should have excl...,"Yes, somewhat rude","No, not at all rude",Once,...,"Yes, very rude","Yes, very rude","Yes, very rude",No,No,Male,30-44,"$25,000 - $49,999",Graduate degree,East North Central
3434235351,Once a month or less,Once in a while,"6'2""",Yes,The person in the middle seat gets both arm rests,Whoever puts their arm on the arm rest first,Everyone in the row should have some say,"No, not rude at all","Yes, somewhat rude",Twice,...,"No, not at all rude","No, not at all rude","Yes, somewhat rude",No,No,Male,30-44,150000,Some college or Associate degree,Pacific
3434218031,Once a year or less,Once in a while,"6'0""",Yes,The arm rests should be shared,The arm rests should be shared,The person in the window seat should have excl...,"No, not rude at all","No, not at all rude",Four times,...,"Yes, somewhat rude","Yes, somewhat rude","Yes, very rude",No,No,Male,30-44,"$0 - $24,999",Bachelor degree,New England
3434213681,Once a year or less,Once in a while,"6'0""",Yes,The arm rests should be shared,The arm rests should be shared,The person in the window seat should have excl...,"No, not rude at all","No, not at all rude",Four times,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
results = json.loads(etiquette_df.T.to_json()).values()

In [15]:
db.etiqueete_metadata.insert_many(results)